## Here we will try our best shot to predict the goals with our features

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Master_UdeM/IFT-6758/Project/ift6758-project-main

/content/gdrive/MyDrive/Master_UdeM/IFT-6758/Project/ift6758-project-main


In [3]:
#!git clone https://GIT_TOKEN@github.com/M0rph3e/ift6758-project-main.git #not anymore
! git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/M0rph3e/ift6758-project-main
   8e853b8..5ed5e8a  best_shot_trial -> origin/best_shot_trial
Already up to date.


In [4]:
!git status

On branch fengineering2
Your branch is up to date with 'origin/fengineering2'.

nothing to commit, working tree clean


In [13]:
!pip install comet_ml pandas_profiling

In [ ]:
! git branch

  best_shot_trial
* fengineering2
  main


# Real **shµt** starts from here

In [5]:
from ift6758.features.feature_engineering2 import SeasonDataSetTwo
from ift6758.data.data_acquisition import Season
from ift6758.visualizations.simple_visualization import Utilities 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import random
import pandas as pd
import pickle
import sys
import json

import warnings
warnings.filterwarnings("ignore")

In [18]:
train_years = [2015,2016,2017]
valid_years = [2018]
test_years=[2019]


train_dataset = SeasonDataSetTwo(train_years)
valid_dataset = SeasonDataSetTwo(valid_years)
train_df = train_dataset.get_features_2()
valid_df = valid_dataset.get_features_2()

File with all events already Exists, loading from ../ift6758/data/PICKLE//2015_clean_all_events.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2015_period_info.pkl
File with all events already Exists, loading from ../ift6758/data/PICKLE//2016_clean_all_events.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2016_period_info.pkl
File with all events already Exists, loading from ../ift6758/data/PICKLE//2017_clean_all_events.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2017_period_info.pkl


100%|██████████| 1299/1299 [02:46<00:00,  7.81it/s]


Len of games_list in 2018 is 1375
Saved new pickle with all events in ../ift6758/data/PICKLE//2018_clean_all_events.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2018.pkl
Len of games_list in 2018 is 1375


In [8]:
test_data = SeasonDataSetTwo(test_years)
test_df = test_data.get_features_2()

100%|██████████| 1299/1299 [02:46<00:00,  7.78it/s]


Len of games_list in 2019 is 1426
Saved new pickle with all events in ../ift6758/data/PICKLE//2019_clean_all_events.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2019.pkl
Len of games_list in 2019 is 1426


# Export the data csv to comet (do not execute for now)

In [149]:
from comet_ml import Experiment
import os



exp = Experiment(
    api_key=os.environ.get('COMET_API_KEY'),
    project_name='milestone-2',
    workspace='morph-e',
)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/morph-e/milestone-2/2689d4e3e26d42a88824134ac2b782bd
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=https%3A%2F%2Fgithub.com%2FM0rph3e%2Fift6758-project-main%2Fblob%2Fbest_shot_trial%2Fnotebooks%2Ftry_your_best_shot.ipynb
COMET INFO:   Parameters:
COMET INFO:     bootstrap                : True
COMET INFO:     ccp_alpha                : 0.0
COMET INFO:     class_weight             : 1
COMET INFO:     copy                     : True
COMET INFO:     criterion                : entropy
COMET INFO:     max_depth                : 10
COMET INFO:     max_features             : auto
COMET INFO:     max_leaf_nodes           : 1
COMET INFO:     max_samples              : 1
COMET INFO:     min_impurity_d

In [22]:
experiment.log_dataframe_profile(
train_df, 
name='train_set_from_FE2',  # keep this name
dataframe_format='csv'  # ensure you set this flag!
)

experiment.log_dataframe_profile(
valid_df, 
name='valid_set_from_FE2',  # keep this name
dataframe_format='csv'  # ensure you set this flag!
)

experiment.log_dataframe_profile(
test_df, 
name='test_set_from_FE2',  # keep this name
dataframe_format='csv'  # ensure you set this flag!
)


COMET WARNING: unable to profile dataframe; ignoring
COMET WARNING: unable to profile dataframe; ignoring
COMET WARNING: unable to profile dataframe; ignoring


{'dataframe': {'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=4e8eecdee4cf45708b03fb750b84e180&experimentKey=2689d4e3e26d42a88824134ac2b782bd',
  'assetId': '4e8eecdee4cf45708b03fb750b84e180',
  'web': 'https://www.comet.ml/api/asset/download?assetId=4e8eecdee4cf45708b03fb750b84e180&experimentKey=2689d4e3e26d42a88824134ac2b782bd'},
 'profile': None}

In [20]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/morph-e/milestone-2/efb5cd930b034742b57308a8f96d6217
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=https%3A%2F%2Fgithub.com%2FM0rph3e%2Fift6758-project-main%2Fblob%2Fbest_shot_trial%2Fnotebooks%2Ftry_your_best_shot.ipynb
COMET INFO:   Uploads:
COMET INFO:     dataframe           : 6 (169.17 MB)
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET INFO: Uploading metrics, params, and assets to Comet before program terminatio

# Split Data

In [46]:
train_df.head()

,result.penaltySeverity,result.penaltyMinutes,gamePk,team.name,gamePeriod,coordinatesX,coordinatesY,shotType,emptyNet,result.strength.name,gameSeconds,shotDistance,shotAngle,isGoal,lastEventType,lastEventCoordinatesX,lastEventCoordinatesY,timeFromLastEvent,distanceFromLastEvent,rebound,speed,changeInShotAngle
0,NaN,NaN,2015020001,Toronto Maple Leafs,1,-55.0,6.0,Wrist Shot,0,NaN,51.0,34.525353,10.007980,0,Hit,94.0,-34.0,11.0,154.275727,False,14.025066,0.000000
1,NaN,NaN,2015020001,Toronto Maple Leafs,1,-79.0,0.0,Snap Shot,0,NaN,65.0,10.000000,0.000000,0,Giveaway,-37.0,-28.0,7.0,50.477718,False,7.211103,0.000000
2,NaN,NaN,2015020001,Toronto Maple Leafs,1,-75.0,-1.0,Snap Shot,0,NaN,66.0,14.035669,-4.085617,0,Shot,-79.0,0.0,1.0,4.123106,True,4.123106,4.085617
3,NaN,NaN,2015020001,Toronto Maple Leafs,1,-55.0,-36.0,Wrist Shot,0,NaN,113.0,49.517674,-46.636577,0,Hit,-91.0,35.0,18.0,79.605276,False,4.422515,0.000000
4,NaN,NaN,2015020001,Toronto Maple Leafs,1,-61.0,-1.0,Tip-In,0,NaN,157.0,28.017851,-2.045408,0,Faceoff,-69.0,-22.0,5.0,22.472205,False,4.494441,0.000000


In [11]:
features = ['gameSeconds','timeFromLastEvent','gamePeriod','shotType', 'shotAngle', 'shotDistance','distanceFromLastEvent', 'rebound','speed','changeInShotAngle']
label = ['isGoal']

In [82]:
train_df[features].isna().any()
valid_df[features].isna().any()
valid_df[features].isna().any()

gameSeconds              False
timeFromLastEvent        False
gamePeriod               False
shotType                 False
shotAngle                False
shotDistance             False
distanceFromLastEvent    False
rebound                  False
speed                     True
changeInShotAngle        False
dtype: bool

### Replace inf with nan (annoying in normalization)

In [81]:
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
valid_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [83]:
#drop rows with nan
train_df = train_df[train_df['shotType'].notna()]
valid_df = train_df[train_df['shotType'].notna()]
test_df = train_df[train_df['shotType'].notna()]

train_df = train_df[train_df['shotAngle'].notna()]
valid_df = train_df[train_df['shotAngle'].notna()]
test_df = train_df[train_df['shotAngle'].notna()]

train_df = train_df[train_df['shotDistance'].notna()]
valid_df = train_df[train_df['shotDistance'].notna()]
test_df = train_df[train_df['shotDistance'].notna()]

train_df = train_df[train_df['speed'].notna()]
valid_df = train_df[train_df['speed'].notna()]
test_df = train_df[train_df['speed'].notna()]

train_df = train_df[train_df['changeInShotAngle'].notna()]
valid_df = train_df[train_df['changeInShotAngle'].notna()]
test_df = train_df[train_df['changeInShotAngle'].notna()]

In [84]:
train_df[features].isna().any()
valid_df[features].isna().any()
valid_df[features].isna().any()

gameSeconds              False
timeFromLastEvent        False
gamePeriod               False
shotType                 False
shotAngle                False
shotDistance             False
distanceFromLastEvent    False
rebound                  False
speed                    False
changeInShotAngle        False
dtype: bool

In [95]:
X_train = train_df[features]
y_train = train_df[label]

X_valid = valid_df[features]
y_valid = valid_df[label]

X_test = test_df[features]
y_test = test_df[label]

X_train, y_train, X_valid, y_valid

(        gameSeconds  timeFromLastEvent  ...      speed changeInShotAngle
 0              51.0               11.0  ...  14.025066          0.000000
 1              65.0                7.0  ...   7.211103          0.000000
 2              66.0                1.0  ...   4.123106          4.085617
 3             113.0               18.0  ...   4.422515          0.000000
 4             157.0                5.0  ...   4.494441          0.000000
 ...             ...                ...  ...        ...               ...
 246798        961.0               20.0  ...   2.960152          0.000000
 246799        966.0                5.0  ...   0.894427          1.582613
 246800        968.0                2.0  ...  26.856098         12.158146
 246801        969.0                1.0  ...   1.414214          0.327134
 246802       1076.0               10.0  ...  15.895911          0.000000
 
 [238046 rows x 10 columns],         isGoal
 0            0
 1            0
 2            0
 3            0
 4

### One Hot encode the shot type (I want to refer on the fact that the shot type affect the goal)

In [96]:
def one_hot(self, y):
        """Return a one hot encoded array of the labels
        """
        return np.eye(len(self.classes))[np.vectorize(lambda c: self.class_labels[c])(y).reshape(-1)]


In [97]:
dum_train = pd.get_dummies(X_train['shotType'], prefix='Shot',dtype=int)
dum_valid = pd.get_dummies(X_valid['shotType'], prefix='Shot',dtype=int)
dum_test = pd.get_dummies(X_test['shotType'], prefix='Shot',dtype=int)

In [98]:
X_train = pd.concat([X_train, dum_train], axis=1)
X_valid = pd.concat([X_valid, dum_valid], axis=1)
X_test = pd.concat([X_test, dum_test], axis=1)

In [99]:
#check if no nans
X_train.isna().any(), X_valid.isna().any(), X_test.isna().any()

(gameSeconds              False
 timeFromLastEvent        False
 gamePeriod               False
 shotType                 False
 shotAngle                False
 shotDistance             False
 distanceFromLastEvent    False
 rebound                  False
 speed                    False
 changeInShotAngle        False
 Shot_Backhand            False
 Shot_Deflected           False
 Shot_Slap Shot           False
 Shot_Snap Shot           False
 Shot_Tip-In              False
 Shot_Wrap-around         False
 Shot_Wrist Shot          False
 dtype: bool, gameSeconds              False
 timeFromLastEvent        False
 gamePeriod               False
 shotType                 False
 shotAngle                False
 shotDistance             False
 distanceFromLastEvent    False
 rebound                  False
 speed                    False
 changeInShotAngle        False
 Shot_Backhand            False
 Shot_Deflected           False
 Shot_Slap Shot           False
 Shot_Snap Shot           F

In [100]:
X_train = X_train.drop(columns='shotType')
X_valid = X_valid.drop(columns='shotType')
X_test = X_test.drop(columns='shotType')

# Change rebound feature

In [101]:
X_train['rebound'] = X_train['rebound'].astype(int)
X_valid['rebound'] = X_valid['rebound'].astype(int)
X_test['rebound'] = X_test['rebound'].astype(int)

In [102]:
X_test.head()

,gameSeconds,timeFromLastEvent,gamePeriod,shotAngle,shotDistance,distanceFromLastEvent,rebound,speed,changeInShotAngle,Shot_Backhand,Shot_Deflected,Shot_Slap Shot,Shot_Snap Shot,Shot_Tip-In,Shot_Wrap-around,Shot_Wrist Shot
0,51.0,11.0,1,10.007980,34.525353,154.275727,0,14.025066,0.000000,0,0,0,0,0,0,1
1,65.0,7.0,1,0.000000,10.000000,50.477718,0,7.211103,0.000000,0,0,0,1,0,0,0
2,66.0,1.0,1,-4.085617,14.035669,4.123106,1,4.123106,4.085617,0,0,0,1,0,0,0
3,113.0,18.0,1,-46.636577,49.517674,79.605276,0,4.422515,0.000000,0,0,0,0,0,0,1
4,157.0,5.0,1,-2.045408,28.017851,22.472205,0,4.494441,0.000000,0,0,0,0,1,0,0


# Normalize numerical data 

In [103]:
num = ['gameSeconds','timeFromLastEvent','gamePeriod', 'shotAngle', 'shotDistance','distanceFromLastEvent','speed','changeInShotAngle']

In [104]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[num])

X_train[num] = scaler.transform(X_train[num])
X_valid[num] = scaler.transform(X_valid[num])
X_test[num] = scaler.transform(X_test[num])

In [107]:
X_test

,gameSeconds,timeFromLastEvent,gamePeriod,shotAngle,shotDistance,distanceFromLastEvent,rebound,speed,changeInShotAngle,Shot_Backhand,Shot_Deflected,Shot_Slap Shot,Shot_Snap Shot,Shot_Tip-In,Shot_Wrap-around,Shot_Wrist Shot
0,-1.560364,-0.307472,-1.220910,0.250264,-0.173438,1.999002,0,0.598526,-0.363797,0,0,0,0,0,0,1
1,-1.520156,-0.578086,-1.220910,-0.024079,-0.944599,-0.250676,0,-0.077369,-0.363797,0,0,0,1,0,0,0
2,-1.517284,-0.984007,-1.220910,-0.136076,-0.817704,-1.255348,1,-0.383676,-0.165387,0,0,0,1,0,0,0
3,-1.382301,0.166103,-1.220910,-1.302502,0.297972,0.380623,0,-0.353977,-0.363797,0,0,0,0,0,0,1
4,-1.255934,-0.713393,-1.220910,-0.080149,-0.378056,-0.857657,0,-0.346842,-0.363797,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246798,1.053136,0.301410,1.146664,0.293374,2.656777,-0.061567,0,-0.499032,-0.363797,0,0,0,0,0,0,1
246799,1.067496,-0.713393,1.146664,0.336757,2.745160,-1.247783,1,-0.703937,-0.286940,0,0,1,0,0,0,0
246800,1.073240,-0.916354,1.146664,0.003473,4.118618,-0.180573,1,1.871270,0.226640,0,0,0,0,0,0,1
246801,1.076112,-0.984007,1.146664,0.012440,4.150697,-1.314060,1,-0.652378,-0.347910,0,0,0,0,0,0,1


## Try Random Forest

In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators':[50,65,80,100,115,130,150],
     'max_depth': [5,6,7,8,9,10,11,12,13,14]
}

rf = RandomForestClassifier(criterion='entropy')
sh = HalvingGridSearchCV(rf,param_grid, cv=4)
model = sh.fit(X_train,y_train)

In [112]:
model.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=115)

In [119]:
y_pred_valid  = model.predict(X_valid)

In [125]:
y_proba_val = model.predict_proba(X_valid)[:,1] # get the proba of having a goal (pred=1)

In [130]:
model.score(X_valid,y_valid)

0.9109415827193064

# Export model on comet before plotting

In [144]:
weights_dir = "../ift6758/models/weights"
experiment = "Random-forest-with-FE2"
predscore_val_dict={}

In [140]:
!ls ..

ift6758  ift6758-project-main


In [145]:
import joblib

predscore_val_dict[experiment]={}
predscore_val_dict[experiment]['true_valid_label']=y_valid
predscore_val_dict[experiment]['proba']=y_proba_val
predscore_val_dict[experiment]['prediction']=y_pred_valid

## Dumping model
filename=f"{weights_dir}/{experiment}.pkl"
joblib.dump(model,filename)

['../ift6758/models/weights/Random-forest-with-FE2.pkl']

In [150]:
exp.log_model("Random Forest", "../ift6758/models/weights/Random-forest-with-FE2.pkl")

{'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=c67d522e79cd4e61a3ead5de12207be8&experimentKey=61ab29d2969f46b3b444cf6aef7b24e6',
 'assetId': 'c67d522e79cd4e61a3ead5de12207be8',
 'web': 'https://www.comet.ml/api/asset/download?assetId=c67d522e79cd4e61a3ead5de12207be8&experimentKey=61ab29d2969f46b3b444cf6aef7b24e6'}

In [151]:
exp.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/morph-e/milestone-2/61ab29d2969f46b3b444cf6aef7b24e6
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=https%3A%2F%2Fgithub.com%2FM0rph3e%2Fift6758-project-main%2Fblob%2Fbest_shot_trial%2Fnotebooks%2Ftry_your_best_shot.ipynb
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model-element       : 1 (9.11 MB)
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET INFO: Uploading 1 metrics, params and output messages
